# 패키지 불러오기, 함수 정의

In [2]:
import os
import pandas as pd
import numpy as np 
#기존에 사용하고 있던 API key!
API_KEY = 

import re


import time
import json
import requests


from sklearn.preprocessing import MinMaxScaler,QuantileTransformer,StandardScaler
import numpy as np
from datetime import datetime


from tqdm import tqdm
tqdm.pandas()
from dotenv import load_dotenv
load_dotenv()

False

In [3]:
def custom_groupby1(df, separator='.'):
    tmp = df.copy(deep=True)
    group_columns = ['Brand Name','Style Code','Style Name']
    join_column = 'Review Text'
    # 그룹화하고 지정된 열에 대해 문자열 연결 수행
    grouped_df = df.groupby(group_columns).agg({join_column: lambda x: separator.join(x.dropna().astype(str))}).reset_index()

    return grouped_df


In [6]:
def keyword(text, retries=3):  # 최대 재시도 횟수를 인자로 받음
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}"
    }

    messages = [
        {"role": "system",
         "content": """
            너는 패션 상품 분석 전문가야. 다음 패션 리뷰 문서를 보고 상품의 특징 5가지를 캐치해줘. 
            일반적으로 잘 나오지 않는 특징을 찾아내야해.리뷰에 있는 단어를 최대한 인용해줘. 단어가 잘리거나 의미가 불완전한 경우, 원래의 문맥을 기반으로 보완하여 완전한 단어로 수정해야 돼. 
            전체적으로 많이 나오는 특징을 잘 나타낸 '단어/문구를 인용'해야돼.

            # Steps
            1. 분석: 리뷰 문서를 분석하여 일반적이지 않은 특징을 파악하세요.
            2. 단어 보정: 키워드가 단독으로 불완전하거나 잘린 경우, 해당 단어가 포함된 문맥을 참고해 원래 의미를 복원하세요. (예: '발편' → '발편함')
            3. 키워드 추출: 일반적이지 않은 각 특징을 적절한 키워드(품사 2개 이상)로 추출하세요.
            4. 키워드 긍부정 추출 : 3번에서 추출된 키워드에 대해 긍정, 부정 평가하세요
			5.언급 빈도 계산: 3번 특징이 리뷰 문서에서 얼마나 등장했는지 계산하세요.
            6. 출처 문장 추출: 각 키워드가 리뷰 문서 내에서 언급된 문장을 확인하여 1개만 출처로 기록하세요.

            # Output Format
            - JSON 형식으로 반환하세요.
            ```json
            {
            "contents": {
                "points": [
                {
                    "point": "키워드",
                    "posi" : "긍정/부정",
                    "frequency": 언급 횟수,
                    "evidence": 출처 문장
                },
                {
                    "point": "키워드",
                    "posi" : "긍정/부정",
                    "frequency": 언급 횟수,
                    "evidence": 출처 문장
                },
                ...
                ]
            }
            }

            # Note
            - 키워드는 5개 이내로 인용
            - '예쁘다', '만족스럽다', '귀엽다','편하다' ,'가성비 좋음','배송빠름', '아이템' 이런 일반적인 특징은 제외하세요
            - 출처 문장이 여러개여도 1개만 제안하시오
            - frequency가 큰 순서대로 추출하시오
            - 긍정, 부정 둘 중 하나로 반환할 것 
         """},
        {"role": "user", "content": f"{text}"}
    ]

    data = {
        "model": "ft:gpt-4o-mini-2024-07-18:eland:week-key-raw:ADqAbJMC",  # 모델 확인
        "messages": messages,
        "temperature": 0.5,
        "max_tokens": 2048,
        "top_p": 1,
        "frequency_penalty": 0.5,
        "presence_penalty": 0.5,
        "stop":["', '"],
        "response_format": {"type": "json_object"}
    }
    
   

    for attempt in range(retries):  # 최대 재시도 횟수만큼 요청 시도
        try:
            response = requests.post(url, headers=headers, json=data)
            response.raise_for_status()  # 요청 에러 발생 시 예외 발생
            
            # JSON 처리
            result = response.json()['choices'][0]['message']['content']
            parsed_result = json.loads(result)  # JSON 파싱
            
            # 'point'와 'evidence' 값을 처리
            points_evidence_pairs = []
            for point_obj in parsed_result.get('contents', {}).get('points', []):  # 'points'로 수정
                point = point_obj.get('point', 'No point')
                posi = point_obj.get('posi','No posi')
                frequency = point_obj.get('frequency', 'No frequency')
                evidence = point_obj.get('evidence', 'No evidence')

                if evidence and evidence.strip() and str(posi) in ['긍정','부정']:
                    points_evidence_pairs.append({
                        'point': point,
                        'posi' : posi,
                        'frequency': frequency,
                        'evidence': evidence
                    })

            return points_evidence_pairs  # 키워드와 근거 문장 쌍을 리스트로 반환

        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"Attempt {attempt + 1} failed: {e}")  # 실패 시 메시지 출력
            time.sleep(2)  # 잠시 대기 후 재시도

    return [] 

# 데이터 불러오기 + 리뷰 분석

In [10]:
#점수 계산 df, 리뷰 df
df_rev = pd.read_excel('D:/업무 파일/한중팀/1.CRD/RAD 버전업/1. 주간/주간_리뷰CS나눠서(1).xlsx')
df_rank = pd.read_excel('D:/업무 파일/한중팀/1.CRD/CRD V2/9월4주차_RANK(1).xlsx')

#태블로로 데이터를 추출했다면
df_rev[['Brand Name','Style Code','Style Name','Category M','Channel']] = df_rev[['Brand Name','Style Code','Style Name','Category M','Channel']].ffill()

#리뷰 텍스트가 없는 행 삭제
df_rev.dropna(subset='Review Text',inplace=True)
df_rev.reset_index(drop=True,inplace=True)

ex_lst = ['광고','체험단','소정의']

for word in ex_lst :
    df_rev = df_rev[~df_rev['Review Text'].str.contains(word,na=False,case=False)]

In [63]:
text = custom_groupby(tmp_text)
text

,Brand Name,Style Code,Style Name,Category M,Review Text
0,MIXXO,MIAABE912A,24mm 반원 벨트,벨트,Review : 가성비 좋아요 바지들이 전부 커서 구매한 벨트입니다 인터넷쇼핑몰벨트...
1,MIXXO,MIAABE916A,18mm 둥근 사각 벨트,벨트,Review : 다른 브라운색보다 좀더 어두운갈색이라 마음에 듭니다~ 23mm 검정...
2,MIXXO,MIAABE920A,27mm 사각 벨트,벨트,Complain : 입고 시 벨트에 구멍 없음.\n\n\nReview : 사이즈가 ...
3,MIXXO,MIAABE922A,18mm 둥근사각 벨트,벨트,Complain : MQ 착장으로 인한 구겨짐
4,MIXXO,MIAAKD917C,크로스 호보백,가방,Review : 여행갈 때 편하게 잘 쓸 거 같아요!
...,...,...,...,...,...
3267,후아유,WHYJE4933F,Steve Denim Shirts(Slim Fit),모시내의,Complain : 곰자수 밑부분 원단 불량 (스티커 참고)\n\n\nReview ...
3268,후아유,WHYWE2511U,Steve Oxford Short Sleeve Shirts,남방,Review : 너무 예쁘네요. 데일리로 다니기에도 너무 편합니다 강추 .색상 너무...
3269,후아유,WHYWE2513F,Steve Crop Short Sleeve Shirts,남방,Review : 생각보다 카라가 뻣벗해서 캐주얼한 느낌보다는 약간 정적인 느낌이 더...
3270,후아유,WHYWE4912F,Steve Oxfort Shirts(Slim Fit),남방,"Complain : 셔츠 뒤쪽 하단 얼룩.(새 상품 뜯은 직후 발견).카라, 넥라인..."


In [65]:
text_sample['point_evidence'] = text_sample['Review Text'].progress_apply(keyword)

# 포인트와 근거를 각각 분리하여 별도의 행으로 저장
expanded_rows = []
for _, row in text_sample.iterrows():
    # row['point_evidence']가 리스트인지 확인
    if isinstance(row['point_evidence'], list):
        for item in row['point_evidence']:
            # item에서 point와 evidence 분리
            point = item.get('point', '없음')  
            positive = item.get('posi','없음')
            freq = item.get('frequency','없음')
            evidence = item.get('evidence', '없음')
            
            expanded_rows.append([
                row['Brand Name'],    
                row['Style Code'],   
                row['Style Name'],  
                row['Review Text'], 
                point,
                positive,                              
                freq,
                evidence  
            ])

df_text = pd.DataFrame(expanded_rows, columns=['브랜드명', '스타일코드', '스타일명', '아이템중분류명', 'Review Text', 'point','posi','freq', 'evidence'])



def remove_punctuation(text):
    for char in ["'", '"', '[', ']']:  
        text = text.replace(char, '')  
    return text


df_text['evidence'] = df_text['evidence'].apply(remove_punctuation)

100%|██████████| 24/24 [01:17<00:00,  3.21s/it]
C:\Users\KIM_MINKYOUNG04\AppData\Local\Temp\ipykernel_32080\384718379.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_sample['point_evidence'] = text_sample['Review Text'].progress_apply(keyword)


In [66]:
df_text

,브랜드명,스타일코드,스타일명,아이템중분류명,Review Text,point,freq,evidence
0,MIXXO,MIWJKET04B,울 라운드넥 자켓,쟈켓,Complain : 단추 두개 떨어짐(주머니 안에 있음).두번째 단추 탈락.세 번...,단정하고 예쁨,3,단정하고 예뻐요 가을에 입기 좋은 색상이에요
1,MIXXO,MIWJKET04B,울 라운드넥 자켓,쟈켓,Complain : 단추 두개 떨어짐(주머니 안에 있음).두번째 단추 탈락.세 번...,팔 길이 수선 예정,1,팔이 길어 수선하려구요
2,MIXXO,MIWJKET04B,울 라운드넥 자켓,쟈켓,Complain : 단추 두개 떨어짐(주머니 안에 있음).두번째 단추 탈락.세 번...,다른 색상 품절,1,다른색상도 구매하고싶은데 품절이네요
3,MIXXO,MIWJKEV66G,스냅 포인트 자켓,쟈켓,Complain : 1번 단추 불량.2번째 단추 뜯어짐 (부자재 동봉).3번째 단추...,유행 안 탈 거,3,유행 안 탈 거 같아서 계속 예쁘게 입을 수 있을 거 같아요오.
4,MIXXO,MIWJKEV66G,스냅 포인트 자켓,쟈켓,Complain : 1번 단추 불량.2번째 단추 뜯어짐 (부자재 동봉).3번째 단추...,질 좋음,2,너무너무 이쁩니다!!!! 질도 좋고 핏도 좋아유 역시 믿고 사는 미쏘.
...,...,...,...,...,...,...,...,...
65,스파오,SPTJE49C51,[데일리지] 와이드 진,청바지,"Complain : 바지 뒷면 기준, 왼쪽 상단에 봉재 서비스 부탁드립니다..앞면 ...",적당한 두께,2,"적당한 두께, 핏 그리고 기장까지 전체적으로 마음에 드네요."
66,스파오,SPTJE49C51,[데일리지] 와이드 진,청바지,"Complain : 바지 뒷면 기준, 왼쪽 상단에 봉재 서비스 부탁드립니다..앞면 ...",평범한 와이드진,2,평범한 와이드진이에요.
67,스파오,SPTJEA9G51,[데일리지] 롱 와이드 진,청바지,Complain : 뒷면 오른쪽 하단에 동그랗게 기름때가 있어 판매가 어려우니 수선...,빈티지 인디고 색깔 짱예쁨,5,빈티지 인디고 색깔 짱예쁨 !!!
68,스파오,SPTJEA9G51,[데일리지] 롱 와이드 진,청바지,Complain : 뒷면 오른쪽 하단에 동그랗게 기름때가 있어 판매가 어려우니 수선...,데일리로 입기 좋은,3,너무 예쁘네요. 데일리로 다니기에도 너무 편합니다 추천 !!


# 점수 계산

In [8]:
df_rank.head()

,스타일코드,스타일명,아이템중분류명,브랜드명,Review Date,Review Text,채널(CS구분)
0,NBPDEF703S,MR530KA,스포츠화,뉴발란스,2024-10-07,기다리고 기다리다 알람받고 바로 구입했어요 배송 아주 빨랐습니다 세번째 530입니다...,리뷰
1,NaN,NaN,NaN,NaN,NaN,매장 입고상품입니다 양발 짝발입니다,CS
2,NaN,NaN,NaN,NaN,NaN,양발 바깥쪽 메쉬터진곳 수선 부탁드립니다.,CS
3,NaN,NaN,NaN,NaN,NaN,양발 발등 앞부분 갑피 매쉬부분 터짐 수선 부탁드립니다,CS
4,NaN,NaN,NaN,NaN,NaN,양발 전체덧댐 힐컵 복원 부탁드립니다,CS


In [11]:
df_rank.rename(columns={'최근 1주 리뷰(COUNTD)':'리뷰수','CS 수 ':'CS 수'},inplace=True)

col_to_int = ['외형매출(기간)','리뷰수','부정 리뷰 수','CS 수']
df_rank[col_to_int] = df_rank[col_to_int].fillna(0).astype(int)


MMscaler = MinMaxScaler()

df_rank['외형매출_조정'] = np.log1p(df_rank[['외형매출(기간)']])


df_rank['CS 수_조정'] = np.log1p(df_rank[['CS 수']])
df_rank[['리뷰수_조정']] =np.log1p(df_rank[['리뷰수']])
df_rank[['부정 리뷰 수_조정']] = np.log1p(df_rank[['부정 리뷰 수']])



df_rank['중요도 점수'] = (df_rank['외형매출_조정']*0.6 +df_rank['리뷰수_조정']*0.4)
df_rank['긴급도 점수'] = (df_rank['부정 리뷰 수_조정']*0.35+df_rank['CS 수_조정']*0.55+df_rank['외형매출_조정']*0.1)

#시즌 가산점 부여
df_rank['중요도 점수_보정'] = df_rank['중요도 점수']

today = datetime.now()
if today > datetime(today.year,8,31) and today <= datetime(today.year,10,31): 
    df_rank.loc[df_rank['시즌코드'].isin(['F', '3']), '중요도 점수_보정'] *= 1.1
elif today > datetime(today.year,10,31) and today <= datetime(today.year,2,28):
    df_rank.loc[df_rank['시즌코드'].isin(['F', '4']), '중요도 점수_보정'] *= 1.1

elif today > datetime(today.year,2,28) and today <= datetime(today.year,5,31):
    df_rank.loc[df_rank['시즌코드'].isin(['S', '1']), '중요도 점수_보정'] *= 1.1
else:
    df_rank.loc[df_rank['시즌코드'].isin(['S', '1']), '중요도 점수_보정'] *= 1.1


final_col = ['최종_중요도', '최종_긴급도']
col_lst = ['중요도 점수', '긴급도 점수'] 
df_rank[final_col] = df_rank[col_lst]

df_rank[final_col] = MMscaler.fit_transform(df_rank[col_lst]) *100

df_rank = df_rank[['스타일코드', '스타일명', '아이템중분류명', '브랜드명', '시즌코드', '외형매출(기간)', '리뷰수', '부정 리뷰 수', 'CS 수', '최종_중요도', '최종_긴급도']]

c:\Users\KIM_MINKYOUNG04\AppData\Local\anaconda3\envs\BD_pj\lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)


# 순위 먹이기

In [12]:
df_rank.head(5)

,스타일코드,스타일명,아이템중분류명,브랜드명,시즌코드,외형매출(기간),리뷰수,부정 리뷰 수,CS 수,최종_중요도,최종_긴급도
0,NBPDEF727S,M1906REH,스포츠화,뉴발란스,F,548627860,75,14,5,100.000000,93.972623
1,NBPDEF704W,MR530SG,스포츠화,뉴발란스,F,450903363,22,1,7,95.684530,80.477616
2,NBPDEF708S,MR530GMT,스포츠화,뉴발란스,F,426587159,36,6,13,96.821044,98.119303
3,NBPDEF724B,M1906RES,스포츠화,뉴발란스,F,304167521,14,2,1,92.735227,64.760252
4,NBPDEF707G,MR530CK,스포츠화,뉴발란스,F,295605505,21,1,11,93.720775,84.783363


In [14]:
#최종 점수 _ 중요도 : 긴급도 = 8:2

df_rank['종합 점수'] = df_rank['최종_중요도'] *0.8 + df_rank['최종_긴급도']*0.2
df_rank['종합 순위'] = df_rank['종합 점수'].rank(ascending=False, method='min')
df_rank['브랜드별 순위'] = df_rank.groupby('브랜드명')['종합 점수'].rank(ascending=False,method='min')

df_rank.loc[df_rank['브랜드명']=='슈펜'].sort_values(by='최종_긴급도',ascending=True,inplace=True)
df_rank

C:\Users\KIM_MINKYOUNG04\AppData\Local\Temp\ipykernel_91704\1063773371.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rank.loc[df_rank['브랜드명']=='슈펜'].sort_values(by='최종_긴급도',ascending=True,inplace=True)


,스타일코드,스타일명,아이템중분류명,브랜드명,시즌코드,외형매출(기간),리뷰수,부정 리뷰 수,CS 수,최종_중요도,최종_긴급도,종합 점수,종합 순위,브랜드별 순위
0,NBPDEF727S,M1906REH,스포츠화,뉴발란스,F,548627860,75,14,5,100.000000,93.972623,98.794525,1.0,1.0
1,NBPDEF704W,MR530SG,스포츠화,뉴발란스,F,450903363,22,1,7,95.684530,80.477616,92.643147,3.0,3.0
2,NBPDEF708S,MR530GMT,스포츠화,뉴발란스,F,426587159,36,6,13,96.821044,98.119303,97.080695,2.0,2.0
3,NBPDEF724B,M1906RES,스포츠화,뉴발란스,F,304167521,14,2,1,92.735227,64.760252,87.140232,14.0,10.0
4,NBPDEF707G,MR530CK,스포츠화,뉴발란스,F,295605505,21,1,11,93.720775,84.783363,91.933292,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6703,NBN0E4A113,UNI IU 월드투어 후드티,스포츠상의,뉴발란스,4,-1281075,1,0,0,NaN,NaN,NaN,NaN,NaN
6704,NK9YE1106U,Essential_ 스케이트보드 일러스트 3PCS 맨투맨 셋업,스포츠상의,뉴발란스 키즈,1,-1288399,2,0,2,NaN,NaN,NaN,NaN,NaN
6705,NBPZES108F,M991BGC,스포츠화,뉴발란스,S,-2849000,0,0,0,NaN,NaN,NaN,NaN,NaN
6706,NKPLEF301G,K920G2P,스포츠화,뉴발란스 키즈,F,-3846126,1,0,0,NaN,NaN,NaN,NaN,NaN


In [21]:
df_rank2=df_rank[df_rank['브랜드명']=='슈펜']
df_rank2.sort_values(by='최종_긴급도',ascending=False,inplace=True)
#df_rank2['최종_긴급도'].value_counts().sum()
df_rank2.shape

C:\Users\KIM_MINKYOUNG04\AppData\Local\Temp\ipykernel_91704\3772649.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rank2.sort_values(by='최종_긴급도',ascending=False,inplace=True)


(424, 14)

# rank, rev 합치기

In [102]:
df = pd.merge(df_rank,df_text,how = 'left', on = ['브랜드명','스타일명','스타일코드'])
df = df[['스타일코드', '스타일명', '브랜드명', '시즌코드', '외형매출(기간)', '리뷰수', '부정 리뷰 수', 'CS 수', '최종_중요도', '최종_긴급도','종합 점수','종합 순위','브랜드별 순위','Review Text','point','evidence']]

In [105]:
df_rank

,스타일코드,스타일명,아이템중분류명,브랜드명,시즌코드,외형매출(기간),리뷰수,부정 리뷰 수,CS 수,최종_중요도,최종_긴급도,종합 점수,종합 순위,브랜드별 순위
0,NBPDEF703S,MR530KA,스포츠화,뉴발란스,F,439582955,73,9,32,100.000000,100.000000,100.000000,1.0,1.0
2,SPFZE4TC01,[리사이클] 베이직 퍼플리스 집업,슬립,SPAO,4,412726400,37,2,15,97.725718,69.660183,92.112611,2.0,1.0
6,NBP7EF754G,ML2002RC,스포츠화,뉴발란스,F,261005633,23,2,15,94.298352,69.660183,89.370719,3.0,2.0
9,NBPDEF714G,MR530TG,스포츠화,뉴발란스,F,205598344,30,1,19,93.993559,68.722771,88.939401,4.0,3.0
4,NBPDEF705W,MR530AD,스포츠화,뉴발란스,F,363334996,28,0,17,96.346640,57.444794,88.566271,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,NBPDES160B,MR530AD,스포츠화,뉴발란스,S,-2064650,14,0,6,NaN,38.674059,NaN,NaN,NaN
2610,NBPZEF755G,U991LG2,스포츠화,뉴발란스,F,-2163000,2,1,0,NaN,9.183985,NaN,NaN,NaN
2611,NBPZEF754B,U991BK2,스포츠화,뉴발란스,F,-2781000,3,1,0,NaN,9.183985,NaN,NaN,NaN
2612,NBPDES166A,MR530KA,스포츠화,뉴발란스,S,-3392702,28,0,17,NaN,57.444794,NaN,NaN,NaN


In [104]:
df.to_excel('D:/업무 파일/한중팀/1.CRD/CRD V2/10월3주차_분석_순위.xlsx')

In [5]:
df = pd.read_csv('C:/Users/KIM_MINKYOUNG04/Downloads/weekly_stylecode_hashtag_03.csv')
df['total_importance_rank'] = df['importance_norm'].rank(method='dense',ascending=False)
df['total_urgency_rank'] = df['urgency_norm'].rank(method='dense',ascending=False)
df['brand_importance_rank'] = df.groupby(['brand'])['importance_norm'].rank(method='dense',ascending=False)
df['brand_urgency_rank'] = df.groupby(['brand'])['urgency_norm'].rank(method='dense',ascending=False)


df.to_csv('C:/Users/KIM_MINKYOUNG04/Downloads/rank추가.csv')


In [ ]:
test4 = pd.read_csv('D:/업무 파일/한중팀/1.CRD/RAD 버전업/1. 주간/4가지케이스테스트.csv')
test4



test4['point_evidence'] = test4['all_review'].progress_apply(keyword)

# 포인트와 근거를 각각 분리하여 별도의 행으로 저장
expanded_rows = []
for _, row in test4.iterrows():
    # row['point_evidence']가 리스트인지 확인
    if isinstance(row['point_evidence'], list):
        for item in row['point_evidence']:
            # item에서 point와 evidence 분리
            point = item.get('point', '없음')  # 만약 key가 없다면 '없음'으로 대체
            freq = item.get('frequency','없음')
            evidence = item.get('evidence', '없음')
            
            expanded_rows.append([
                row['style_code'],   # 추가된 부분
                row['all_review'], # 원본 리뷰
                point,              # 차별화 포인트
                freq,
                evidence   # 포맷된 근거 문장
            ])

df_test4 = pd.DataFrame(expanded_rows, columns=['style_code','all_review', 'point','freq', 'evidence'])

In [ ]:
import pandas as pd
import json

# 엑셀 파일 로드
df = pd.read_excel("D:/업무 파일/한중팀/1.CRD/RAD 버전업/2. 상세_시즌/상세_긍부정테스트.xlsx")  # 파일명을 올바르게 변경해주세요

# JSONL 형식으로 변환 및 저장
with open("output.jsonl", "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        data = {
            "messages": [
                {"role": "system", "content": row["prompt"]},
                {"role": "user", "content": row["User content"]},
                {"role": "assistant", "content": row["Assistant content"]}
            ]
        }
        json.dump(data, f, ensure_ascii=False)
        f.write("\n")
